# Monte-Carlo Simulation

Import necessary packages

In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
import openmm
import numpy as np
import tensorflow as tf
from tensorflow import keras
import scipy.constants as sc
from matplotlib import pyplot as plt
import os
from IPython.display import clear_output
from scipy.interpolate import RegularGridInterpolator
import MDAnalysis as mda
from MDAnalysis.analysis import rms
import jupyter_beeper
beep = jupyter_beeper.Beeper()
import copy

Define system name and system-specific functions

In [ ]:
system_name = "trpcage"

In [ ]:
os.system("pwd")

In [ ]:
cpu_platform = Platform.getPlatformByName('CPU')
cuda_platform = Platform.getPlatformByName('CUDA')

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
model_name = "n_5e-2_4"
decoder_name = "d_"+model_name
encoder_name = "e_"+model_name

In [ ]:
def Standardize(x):
    x = np.array(x)
    if system_name == "trpcage":
        result = (x + 1.0)/6
    elif system_name == "villin":
        result = (x + 2.0)/9
    elif system_name == "pdz":
        result = (x - 1.5)/9
    return result

def Unstandardize(x):
    x = np.array(x)
    if system_name == "trpcage":
        result = (x * 6) + 1.0
    elif system_name == "villin":
        result = (x * 9) - 2.0
    elif system_name == "pdz":
        result = (x * 9) + 1.5
    return result

Define MCMC acceptance function

In [ ]:
def MCAccept(e1, e2, temp):
    try:
        if e2.value_in_unit(kilojoule/mole) < -1e5:
            print("Warning: Pot. E. attempt under -1e+5\n")
            return False
    except:
        ...

    try:
        e2 = e2.value_in_unit(joule/mole)
    except:
        ...
    try:
        e1 = e1.value_in_unit(joule/mole)
    except:
        ...
        
    deltaE = e2 - e1
    
    if deltaE < 0:
        return True
    else:
        boltzmann_factor = np.exp(-deltaE / (8.314 * temp))
        rn = np.random.uniform()
        #print(f"dE = {deltaE}, BF = {boltzmann_factor}, random number = {rn}, ", end="\t")
        if  rn < boltzmann_factor:
            return True
        else:
            return False

Custom potential energy minimization procedure

In [ ]:
class GradientDescentMinimizationIntegrator(CustomIntegrator):

    """Simple gradient descent minimizer implemented as an integrator.

    Examples
    --------

    Create a gradient descent minimization integrator.

    >>> integrator = GradientDescentMinimizationIntegrator()

    """

    def __init__(self, initial_step_size=0.01 * angstroms, p1 = 1.2, p2 = 0.5):
        """
        Construct a simple gradient descent minimization integrator.
    
        Parameters
        ----------
        initial_step_size : np.unit.Quantity compatible with nanometers, default: 0.01*unit.angstroms
           The norm of the initial step size guess.
    
        Notes
        -----
        An adaptive step size is used.
    
        """
    
        timestep = 1.0 * femtoseconds
        super(GradientDescentMinimizationIntegrator, self).__init__(timestep)

        self.addGlobalVariable("step_size", initial_step_size / nanometers)
        self.addGlobalVariable("energy_old", 0)
        self.addGlobalVariable("energy_new", 0)
        self.addGlobalVariable("delta_energy", 0)
        self.addGlobalVariable("accept", 0)
        self.addGlobalVariable("fnorm2", 0)
        self.addPerDofVariable("x_old", 0)
        
        self.addGlobalVariable("p1", p1)
        self.addGlobalVariable("p2", p2)

        # Update context state.
        self.addUpdateContextState()

        # Constrain positions.
        self.addConstrainPositions()
        
        # Store old energy and positions.
        self.addComputeGlobal("energy_old", "energy")
        self.addComputePerDof("x_old", "x")

        # Compute sum of squared norm.
        self.addComputeSum("fnorm2", "f^2")

        # Take step.
        self.addComputePerDof("x", "x+step_size*0.1*f/(sqrt(fnorm2 + delta(fnorm2)))")
        self.addComputeSum("fnorm2", "f^2")

        # Take step.
        self.addComputePerDof("x", "x+step_size*0.2*f/(sqrt(fnorm2 + delta(fnorm2)))")
        self.addComputeSum("fnorm2", "f^2")

        # Take step.
        self.addComputePerDof("x", "x+step_size*0.3*f/(sqrt(fnorm2 + delta(fnorm2)))")
        self.addComputeSum("fnorm2", "f^2")

        # Take step.
        self.addComputePerDof("x", "x+step_size*0.4*f/(sqrt(fnorm2 + delta(fnorm2)))")
        self.addComputeSum("fnorm2", "f^2")

        # Take step.
        self.addComputePerDof("x", "x+step_size*0.5*f/(sqrt(fnorm2 + delta(fnorm2)))")
        self.addConstrainPositions()

        # Ensure we only keep steps that go downhill in energy.
        self.addComputeGlobal("energy_new", "energy")
        self.addComputeGlobal("delta_energy", "energy_new-energy_old-0.00000001")
        # 
        self.addComputeGlobal("accept", "step(-delta_energy) * delta(energy - energy_new)")

        self.addComputePerDof("x", "accept*x + (1-accept)*(x_old+0.0001*(step(f)-step(-f)))")

        # Update step size.
        self.addComputeGlobal("step_size", "step_size * (p1*accept + p2*(1-accept))")

## Load the trained decoder

In [ ]:
#load decoder model
decoder = tf.keras.models.load_model("../ae/models/"+decoder_name+".keras")
#decoder = tf.keras.models.load_model('tarkil_training/models/decoder_mse_then_dist')

Initialize OpenMM simulation systems

In [ ]:
output_trj = "output_"+system_name+"_"+model_name+".pdb"

pdb = PDBFile("../data/"+system_name+"_reference.pdb")

forcefield = ForceField('amber14-all.xml', 'implicit/gbn2.xml')

#modeller = Modeller(pdb.topology, pdb.positions)
#modeller.addHydrogens(forcefield)

In [ ]:
system = forcefield.createSystem(pdb.topology, nonbondedMethod=NoCutoff)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)#0.002*picoseconds)
#simulation = Simulation(pdb.topology, system, integrator, platform=cuda_platform)
simulation = Simulation(pdb.topology, system, integrator, platform=cpu_platform)
simulation.reporters.append(PDBReporter(output_trj, 500))

# system for evaluations of potential enegrgy
eval_system = forcefield.createSystem(pdb.topology,nonbondedMethod=NoCutoff)
eval_integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)#0.002*picoseconds)
eval_simulation = Simulation(pdb.topology, eval_system, eval_integrator)

mels_system = forcefield.createSystem(pdb.topology, nonbondedMethod=NoCutoff)
mels_integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
mels_simulation = Simulation(pdb.topology, mels_system, mels_integrator, platform=cpu_platform)
mels_simulation.reporters.append(PDBReporter(f"mels_{output_trj}", 500))

In [ ]:
# initial structure with topology from reference
simulation.context.setPositions(pdb.positions)
state = simulation.context.getState(getEnergy=True, getPositions=True)
print(state.getPotentialEnergy())
simulation.reporters[0].report(simulation, state)

In [ ]:
# initial structure from native decoded structure instead
latent_space = np.load("../ae/models/encoded_reference_"+model_name + ".npy") # folded
#latent_space = np.load("tarkil_training/encoded_fold_decoder_mse_then_dist.npy")
#latent_space = np.random.normal(size = ((1, 64)))
# latent_space = np.array([[ -1.8, -3.2,  0.2, -0.6,  0.6, 1.8, -1.5, 0.3]]) # unfolded
#latent_space = Rmsd_ls(latent_space = latent_space, histograms = histograms, rmsd=0.12)
#latent_space = np.array([[-1.0,+1.0]])

pos = decoder(latent_space).numpy().transpose()
#pos = np.load("some_structure.npy")
print(latent_space.shape)
print(pos.shape)
pos = Unstandardize(pos.reshape((int(pos.shape[0]/3), 3)).tolist())

In [ ]:
%%time
simulation.context.setPositions(pos)
simulation.minimizeEnergy(maxIterations=200, tolerance = 4.0e2)
state = simulation.context.getState(getEnergy=True, getPositions=True)
E = state.getPotentialEnergy()
#simulation.reporters[0].report(simulation, state)
E = E.value_in_unit(joule/mole)
print(E/1000)

## Precalculate the protein structures on latent space

Define the matrix on the latent space to precalculate protein structures

In [ ]:
matrix_res = 64

In [ ]:
LS_matrix = np.zeros((matrix_res,matrix_res,latent_space.shape[1]))
LS_matrix_pos = np.zeros((matrix_res,matrix_res, pos.shape[0]*pos.shape[1]))
LS_matrix_pos_min = np.zeros((matrix_res,matrix_res, pos.shape[0]*pos.shape[1]))
LS_matrix_E = np.zeros((matrix_res,matrix_res))
LS_matrix_E_min = np.zeros((matrix_res,matrix_res))

In [ ]:
%%time
# calculate pot. E for predicted structures
m_energies_list = []
frame = 1

# system for matrix frames
custom_force = CustomNonbondedForce("step(tr - r) * k * (tr - r)^2")
custom_force.addGlobalParameter("tr", 0.095 * nanometers)
custom_force.addGlobalParameter("k", 100000.0)
custom_force.addInteractionGroup(range(pdb.getPositions(asNumpy=True)._value.shape[0]),range(pdb.getPositions(asNumpy=True)._value.shape[0]))

m_system = forcefield.createSystem(pdb.topology)

for _ in range(pdb.getPositions(asNumpy=True)._value.shape[0]):
    custom_force.addParticle(())

m_system.addForce(custom_force)

m_integrator = GradientDescentMinimizationIntegrator(initial_step_size=1e3 * angstroms, p1=1.01, p2=0.5)
m_simulation = Simulation(pdb.topology, m_system, m_integrator, platform=cuda_platform)
m_simulation.reporters.append(PDBReporter("matrix_"+str(matrix_res)+"_"+output_trj, 1e10))

m_simulation.context.setPositions(pdb.positions)
m_state = simulation.context.getState(getEnergy=True, getPositions=True)
#print(state.getPotentialEnergy())
m_simulation.reporters[0].report(m_simulation, m_state)

lbfgs_system = forcefield.createSystem(pdb.topology)
lbfgs_integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
lbfgs_simulation = Simulation(pdb.topology, lbfgs_system, lbfgs_integrator)

try: 
    LS_matrix_pos = np.load(f"LS_matrix_{matrix_res}_pos.npy")
    print("Loaded decoder outputs\n")
except:
    for i in range(matrix_res):
        LS_matrix[i,:,0] = -1.25+2.5*i/matrix_res
    for i in range(matrix_res):
        LS_matrix[:,i,1] = -1.25+2.5*i/matrix_res
    # encoder predicts atom positions
    for i in range(matrix_res):
        print(i, end="\r")
        for j in range(matrix_res):
            LS_matrix_pos[i,j,:] = decoder(LS_matrix[i,j,:].reshape((1,2))).numpy().reshape(pos.shape[0]*pos.shape[1])
    np.save(f"LS_matrix_{matrix_res}_pos.npy", LS_matrix_pos)
try:
    LS_matrix_E_min = np.load(f"LS_matrix_{matrix_res}_E_min.npy")
    LS_matrix_pos_min = np.load(f"LS_matrix_{matrix_res}_pos_min.npy")
except:
    for i in range(matrix_res):
        #print(i, end="\r")
        for j in range(matrix_res):
            #try:
            pos = LS_matrix_pos[i,j,:]
            pos = Unstandardize(pos.reshape((int(pos.shape[0]/3), 3)))
            m_simulation.context.reinitialize()
            m_simulation.context.setPositions(pos)
            m_simulation.integrator.setGlobalVariableByName('step_size', 1e2)
            
            for s in range(1000):
                m_simulation.step(1)
                m_state = m_simulation.context.getState(getForces=True, getEnergy=True)
                m_force_components = m_state.getForces(asNumpy=True)._value
                m_max_force = m_force_components.max()
                if m_max_force < 400:
                    break
            if m_max_force >= 400:
            # if Steepest Descent fails to minimize, try short L-BFGS and thes SD again
                print("Trying L-BFGS... ", end = "")
                lbfgs_simulation.context.reinitialize()
                lbfgs_simulation.context.setPositions(pos)
                lbfgs_simulation.minimizeEnergy(maxIterations=100, tolerance=1e4)
                lbfgs_state = lbfgs_simulation.context.getState(getPositions=True, getEnergy=True)
                lbfgs_positions = lbfgs_state.getPositions()
                print(f"energy: {lbfgs_state.getPotentialEnergy()}, switching back to SD")
                m_simulation.context.reinitialize()
                m_simulation.context.setPositions(lbfgs_positions)
                m_simulation.integrator.setGlobalVariableByName('step_size', 1e2)
                
                for s in range(1000):
                    m_simulation.step(1)
                    m_state = m_simulation.context.getState(getForces=True, getEnergy=True)
                    m_force_components = m_state.getForces(asNumpy=True)._value
                    m_max_force = m_force_components.max()
                    if m_max_force < 400:
                        break
                        
            m_state = m_simulation.context.getState(getEnergy=True, getPositions=True)
            m_E = m_state.getPotentialEnergy()
            m_simulation.reporters[0].report(m_simulation, m_state)
            
            LS_matrix_pos_min[i,j,:] = np.array(m_state.getPositions()._value).reshape(pos.shape[0]*pos.shape[1])
            LS_matrix_E_min[i,j] = m_E.value_in_unit(joule/mole)
            print(f"{i},{j}, energy: {m_E.value_in_unit(kilojoule/mole)}, frame = {frame}.")#, end="\r")
            frame += 1
            m_energies_list.append(m_E.value_in_unit(kilojoule/mole))
            #except:
                #print(f"{i},{j} Openmm minimization failed.", end="")
                ##m_E = m_state.getPotentialEnergy()
                #m_simulation.context.setPositions(pos)
                #m_state = m_simulation.context.getState(getPositions=True)
                #m_simulation.reporters[0].report(m_simulation, m_state)
                #LS_matrix_pos_min[i,j,:] = np.array(m_state.getPositions()._value).reshape(pos.shape[0]*pos.shape[1])
                
                #LS_matrix_E_min[i,j] = 1e10#m_E.value_in_unit(joule/mole)
                #print(f"{i},{j}, energy (---- failed ----): {1e10}, frame = {frame}.")#, end="\r")
                #frame += 1
                #m_energies_list.append(1e10)

    
    beep.beep(secs=0.3, blocking=True)
    beep.beep(secs=0.5)
    np.save(f"LS_matrix_{matrix_res}_E_min.npy", LS_matrix_E_min)
    np.save(f"LS_matrix_{matrix_res}_pos_min.npy", LS_matrix_pos_min)

In [ ]:
np.min(LS_matrix_E_min)/1000, np.max(LS_matrix_E_min)/1000

Loop up where on the latent space are structures with lowest potential energies (This also shows potentional errors in the energy minimization). 

In [ ]:
# ith lowest value is where?
for i in range(10):
    ii = np.where(np.sort(LS_matrix_E_min.ravel())[i] == LS_matrix_E_min)[0][0]
    ij = np.where(np.sort(LS_matrix_E_min.ravel())[i] == LS_matrix_E_min)[1][0]
    print(ii, ij, LS_matrix_E_min[ii,ij]/1000)

If any potential energies are lower than -5 MJ/mol, this is likely due to an error in the calculation:

In [ ]:
LS_matrix_E_min[LS_matrix_E_min < -5e6] = -LS_matrix_E_min[LS_matrix_E_min < -5e6]

In [ ]:
LS_matrix_E_min = np.nan_to_num(LS_matrix_E_min, nan=1e8)

In [ ]:
# ith lowest value is where?
for i in range(10):
    ii = np.where(np.sort(LS_matrix_E_min.ravel())[i] == LS_matrix_E_min)[0][0]
    ij = np.where(np.sort(LS_matrix_E_min.ravel())[i] == LS_matrix_E_min)[1][0]
    print(ii, ij, LS_matrix_E_min[ii,ij]/1000)

Plot the potential energies of decoded and minimized structures

In [ ]:
plt.imshow(np.rot90(LS_matrix_E_min/1000, axes=(0,1)), vmax=np.min(LS_matrix_E_min/1000)+5e3, vmin=None, cmap="viridis", extent=[-1.25, 1.25, -1.25, 1.25])
plt.xlabel("latent value 1")
plt.ylabel("latent value 2")
plt.colorbar()

Calculate the RMSD of decoded and minimized structures from the native structure

In [ ]:
try:
    m_rmsd = np.load(f"m_rmsd_{matrix_res}.npy")
except:
    m_u = mda.Universe("matrix_"+str(matrix_res)+"_"+output_trj)
    u_ref = mda.Universe("../data/"+system_name+"_reference.pdb")
    m_u.trajectory
    m_u.trajectory[0] # set to first frame
    m_u_rmsd = rms.RMSD(m_u, u_ref, select='name CA')
    m_u_rmsd.run()
    m_rmsd = m_u_rmsd.rmsd[1:,2]/10
    m_rmsd = m_rmsd.reshape((matrix_res, matrix_res))
    np.save(f"m_rmsd_{matrix_res}.npy", m_rmsd)

#m_rmsd = 
plt.imshow(np.rot90(m_rmsd, axes=(0,1)), vmax=None, cmap="viridis", extent=[-1.25, 1.25, -1.25, 1.25])
cont = plt.contour(np.flip(np.rot90(m_rmsd, axes=(1,0)), axis=1), 
                   levels=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5,1.6], 
                   extent=[-1.25, 1.25, -1.25, 1.25], cmap="jet")
plt.xlabel("latent value 0")
plt.ylabel("latent value 1")
plt.colorbar()

Prepare the continuous potential energy surface by interpolation

In [ ]:
lspace = np.linspace(-1.25,1.25, matrix_res, endpoint=False)
fit_points = [lspace,lspace]
rgi = RegularGridInterpolator(fit_points, LS_matrix_E_min, method="linear")
interpolate_grid = np.mgrid[-1.25:(1.25-3/matrix_res):0.005, -1.25:(1.25-3/matrix_res):0.005]
interpolated = np.zeros((interpolate_grid.shape[1:]))
for i in range(interpolate_grid.shape[-1]):
    for j in range(interpolate_grid.shape[-1]):
        interpolated[i,j] = rgi([interpolate_grid[0,i,j], interpolate_grid[1,i,j]])[0]

Plot the continuous potential energy surface

In [ ]:
plt.imshow(np.rot90(interpolated/1000, axes=(0,1)), vmax=0, cmap="RdYlBu_r", extent=[-1.25, 1.25, -1.25, 1.25])
plt.xlabel("latent value 1")
plt.ylabel("latent value 2")
#plt.title("Potential energy of latent space")
cb = plt.colorbar()
cb.set_label("potential energy (kJ/mol)")
plt.savefig("PE_LS.png", dpi=600)
plt.show()

## Perform the MCMC simulation

Initialize the beginning of the MCMC simulation

 - save starting structure and energy
 - Set parameters $f$ to `HYP_F` and $d$ to `HYP_D`
 - Load reference RMSD distribution for comparison, if the reference data are available

In [ ]:
continuation=False
if continuation:
    try:
        accepted_mels = np.load("accepted_mels.npy")
        print(f"Loaded previous latent space history with lenght {accepted_mels.shape[0]}")
        latent_space = accepted_mels[-1,:] 
    except:
        print("Error: Previous output not loaded.")
else:
    accepted_mels = latent_space

accepted = 0
attempts = int(5e6)
eval_latent_space = latent_space
latent_space_bins = ((latent_space+1.25)*matrix_res/2.5).astype(int)

E = rgi(latent_space)[0]
accepted_energies = [E]
accepted_energies_mels = [E]
acceptance = [1]
acc_rmsd_weights = [1]
eval_E = E

ls_list = latent_space

eval_history = []
mels = latent_space
accepted_rmsd = [m_rmsd[latent_space_bins[0][0], latent_space_bins[0][1]]]
accepted_rmsd_mels = [m_rmsd[latent_space_bins[0][0], latent_space_bins[0][1]]]
accepted_mels = latent_space

HYP_F = 200
HYP_D = 0.001
try: # normally load rmsd values from D.E.Shaw MD trajectory - reference distribution; for pdz it is not available
    ref_rmsd = np.loadtxt("../data/"+system_name+"_rmsd_reference.xvg")
except:
    ref_rmsd = np.loadtxt("../data/"+system_name+"_rmsd_ds.xvg")
bins = 50
refpop = np.histogram(ref_rmsd[:,1], bins=bins, range=(0, 2.0), density=True)
#eval_latent_space = np.array([[ , 0]]) 
#eval_ls_bins = ((eval_latent_space+1.5)*matrix_res/3).astype(int)
u = mda.Universe("../data/"+system_name+"_reference.pdb")

Perform the MCMC simulation
Visualization includes

- Potential energy surface with the MCMC trajectory over it
- Free energy surface with RMSD from reference structure as a CV calculated from the MCMC simulation on-the-run
- C alpha atoms of the protein visualized in 3D at the current MCMC step

In [ ]:
%%time
for i in range(0,attempts):   
    # write report about progress
    if ((i) % 50000 == 0) or (i == attempts-1):
        print(f"Progress: {((i)/(attempts-1)):.1%}, accepted so far: {np.sum(acceptance)}, acceptance ratio: {np.mean(acceptance):.3%}, last energy: {(eval_E/1000):.2e} kJ/mol (accepted: {(E/1000):.2e} kJ/mol).", end="\r")
    
    # get new latent space values
    if i % int(HYP_F) == 0:
        # uniform - direction
        angle = np.random.uniform(low=0, high=2*np.pi, size=1)
        master_eval_latent_space = latent_space + np.array((np.cos(angle), np.sin(angle))).T*0.1
        while(np.any(master_eval_latent_space > 1.25-2.5/matrix_res-1e-7) or np.any(master_eval_latent_space < -1.25)):
            angle = np.random.uniform(low=0, high=2*np.pi, size=1)
            master_eval_latent_space = latent_space + np.array((np.cos(angle), np.sin(angle))).T*0.1
        latent_space_before = latent_space
        latent_space_bins = ((latent_space+1.25)*matrix_res/2.5).astype(int)
        
        # Sampling last accepted structure every "HYP_F" attempts
        # save the energy
        accepted_energies.append(E)
        accepted_mels = np.vstack((accepted_mels, latent_space))
        
        accepted_rmsd.append(m_rmsd[latent_space_bins[0][0],latent_space_bins[0][1]])

        # save the structure into trajectory file
        mels_simulation.context.setPositions(LS_matrix_pos_min[latent_space_bins[0][0], latent_space_bins[0][1], :].reshape(pos.shape))
        mels_state = mels_simulation.context.getState(getPositions=True)
        mels_simulation.reporters[0].report(mels_simulation, mels_state)

    eval_latent_space = np.clip(latent_space +
                            HYP_D*10*(master_eval_latent_space - latent_space_before), 
                                -1.25, 1.25-2.5/matrix_res-1e-7)
    eval_E = rgi([eval_latent_space[0,0],eval_latent_space[0,1]])[0]
    
    # monte carlo
    if MCAccept(E, eval_E, 290.0):
        latent_space = eval_latent_space
        E = eval_E
        accepted += 1
        acceptance.append(1)
    else:
        acceptance.append(0)

    # visual output
    if i % 50000 == 0:
        clear_output(wait=True)
        
        #pop = np.histogram(accepted_rmsd[:], bins=bins, range=(0, 2.0), density=True, weights = acc_rmsd_weights)
        pop_mels = np.histogram(accepted_rmsd_mels[:], bins=bins, range=(0, 2.5), density=True)
        reftemp = 290
        temp = 290
        
        fig, axes = plt.subplots(1, 3, figsize = (18,4))
        #plt.tight_layout()
        axes[1].plot(refpop[1][:-1], (-np.log(refpop[0])*8.314*reftemp/1000)-np.min((-np.log(refpop[0])*8.314*reftemp/1000)))
        axes[1].plot(refpop[1][:-1], +4.18+(-np.log(refpop[0])*8.314*reftemp/1000)-
                     np.min((-np.log(refpop[0])*8.314*reftemp/1000)), color="lightblue")
        axes[1].plot(refpop[1][:-1], -4.18+(-np.log(refpop[0])*8.314*reftemp/1000)-
                     np.min((-np.log(refpop[0])*8.314*reftemp/1000)), color="lightblue")
        #axes[1].plot(pop[1][:-1], (-np.log(pop[0])*8.314*temp/1000)-np.min((-np.log(pop[0])*8.314*temp/1000)))
        axes[1].plot(pop_mels[1][:-1], (-np.log(pop_mels[0])*8.314*temp/1000)-np.min((-np.log(pop_mels[0])*8.314*temp/1000)), 
                     color="orange")
        axes[1].set_ylabel("free energy (kJ/mol)")
        axes[1].set_xlim((0,2.0))
        
        img = axes[0].imshow(np.rot90(interpolated/1000, axes=(0,1)), vmax=0, 
                             cmap="viridis", extent=[-1.25, 1.25, -1.25, 1.25])
        axes[0].contour(np.flip(np.rot90(m_rmsd, axes=(1,0)), axis=1),
                        levels=[0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6], 
                        extent=[-1.25, 1.25, -1.25, 1.25], cmap="jet")
        axes[0].set_xlabel("latent value 1")
        axes[0].set_ylabel("latent value 2")
        cbar = plt.colorbar(img, ax=axes[0])
        cbar.set_label("potential energy")
        axes[0].scatter(accepted_mels[:-500,0], accepted_mels[:-500,1], c="lightgray", s=1)
        axes[0].scatter(accepted_mels[-500:-100,0], accepted_mels[-500:-100,1], c="red", s=1)
        axes[0].plot(accepted_mels[-100:,0], accepted_mels[-100:,1], c="black")

        axes[2] = fig.add_subplot(1, 3, 3, projection='3d')
        u.atoms.positions = LS_matrix_pos_min[latent_space_bins[0][0], latent_space_bins[0][1], :].reshape(pos.shape)
        axes[2].plot3D(u.select_atoms("name CA").positions[:, 0], 
                       u.select_atoms("name CA").positions[:, 1], 
                       u.select_atoms("name CA").positions[:, 2])
        plt.show()
print(f"\nAccepted: {np.sum(acceptance)}")

Save the MCMC simulation progress

In [ ]:
np.save("accepted_mels.npy", accepted_mels)
accepted_mels.shape, HYP_F, HYP_D

## Analyse the MCMC results

Calculate and plot the Free energy surface calculated using MCMC samples over the latent space

In [ ]:
hist2d = np.histogram2d(accepted_mels[:,0], accepted_mels[:,1], bins=32, range=[[-1.25,1.25],[-1.25, 1.25]], density=True)

In [ ]:
plt.imshow(np.rot90((-np.log(hist2d[0])*8.314*temp/1000)-
                    np.min((-np.log(hist2d[0])*8.314*temp/1000)), axes=(0,1)), 
           cmap="RdYlBu_r", extent=[-1.25, 1.25, -1.25, 1.25])
plt.xlabel("latent value 1")
plt.ylabel("latent value 2")
plt.colorbar()

Plot the potential energy surface and the MCMC trajectory

In [ ]:
plt.imshow(np.rot90(LS_matrix_E_min/1000, axes=(0,1)), vmax=0, cmap="RdYlBu_r", extent=[-1.25, 1.25, -1.25, 1.25])
plt.xlabel("latent value 1")
plt.ylabel("latent value 2")
plt.colorbar()
plt.plot(accepted_mels[::1,0], accepted_mels[::1,1], c="black")

In [ ]:
plt.imshow(np.rot90(interpolated/1000, axes=(0,1)), vmax=0, cmap="RdYlBu_r", extent=[-1.25, 1.25, -1.25, 1.25])
plt.xlabel("latent value 1")
plt.ylabel("latent value 2")
#plt.title("Potential energy of latent space")
cb = plt.colorbar()
cb.set_label("potential energy (kJ/mol)")
plt.plot(accepted_mels[::50,0], accepted_mels[::50,1], c="black")
plt.savefig("PE_LS_MC.png", dpi=600)

Plot the potential energies of structures sampled during MCMC simulation

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(range(len(accepted_energies_mels)), np.array(accepted_energies_mels[:])/1000)
ax.set_xlabel("MCMC samples")
ax.set_ylabel("potential energy (kJ/mol)")

plt.savefig("PE_300.png", dpi=600)

Plot the RMSD from reference structure of the samples from MCMC simulation

In [ ]:
u = mda.Universe("mels_"+output_trj)
u_ref = mda.Universe("../data/"+system_name+"_reference.pdb")
print(u.trajectory)
u.trajectory[0] # set to first frame
u_rmsd = rms.RMSD(u, u_ref, select='name CA')
u_rmsd.run()
u_rmsd = u_rmsd.results.rmsd[1:,2]/10

plt.figure(figsize=((10,5)))
plt.plot(u_rmsd)
plt.ylim((0,1.2))
plt.xlabel("MCMC samples")
plt.ylabel("RMSD (nm)")
plt.savefig(system_name+"_rmsd.png", dpi=600)

Plot the average acceptance ratio during MCMC simulation

In [ ]:
plt.figure(figsize=(10,2))
plt.scatter(range(0,len(acceptance)), acceptance[:], s = 1)

window = 100
average_ac = []
for ind in range(int(len(acceptance)/window + 1)):
    average_ac.append(np.mean(acceptance[ind*window:ind*window+window]))
print("plotting")
plt.scatter(range(0,len(average_ac)*window, window), average_ac[:], s = 1, color ="orange")
plt.xlabel("steps")
plt.ylabel("ratio")

Calculate the Free energy surface from the sampled population of structures and compare it to a reference population, if available

In [ ]:
bins = 32
refpop = np.histogram(ref_rmsd[:,1], bins=bins, range=(0, 1.5), density=True)
pop = np.histogram(accepted_rmsd_mels[:], bins=bins, range=(0, 1.5), density=True, weights = None)
reftemp = 290
temp = 290

fig, axes = plt.subplots(1, 3, figsize=(16,4))
axes[0].bar(refpop[1][:-1], refpop[0]/np.sum(refpop[0]), width=1.3/bins)
axes[0].set_title("Molecular dynamics (D.E.Shaw)")
axes[0].set_xlabel("RMSD (nm)")
axes[0].set_ylabel("relative population")

axes[1].bar(pop[1][:-1], pop[0]/np.sum(pop[0]), color="orange", width=1.3/bins)
axes[1].set_title("MCMC with autoencoder")
axes[1].set_xlabel("RMSD (nm)")

axes[2].plot(refpop[1][:-1], (-np.log(refpop[0])*8.314*reftemp/1000)-np.min((-np.log(refpop[0])*8.314*reftemp/1000)))
axes[2].plot(pop[1][:-1], (-np.log(pop[0])*8.314*temp/1000)-np.min((-np.log(pop[0])*8.314*temp/1000)))
axes[2].set_title("Free energy comparison")
axes[2].set_ylabel("free energy (kJ/mol)")
axes[2].set_xlabel("RMSD (nm)")
axes[2].set_xlim((0,1.5))
plt.savefig(system_name+"_MC_fes.png", dpi=600, bbox_inches = 'tight')